$$- \nabla \cdot (a(x, \omega_1) \cdot \nabla u(x)) = f(x) \hspace{0.2cm} \text{on} \hspace{0.2cm} D(\omega_2)$$
where $a(x, \omega_1) = \exp(Z(x, \omega_1))$, $Z$ being gaussian  \
and $V(x, \omega_2)$ is the random field corresponding to $D(\omega_2)$

In [1]:
from helpers_diffusion import *
from helpers import *

In [2]:
MESH_RESOLUTION = 4

In [ ]:
# for MESH_RESOLUTION = 4 it takes approx. 12 seconds

# Calculate the eigenpairs
randomFieldV, jacobianV = calculate_vector_field_eigenpairs(MESH_RESOLUTION)

In [5]:
randomFieldZ = z_calculate_random_field_eigenpairs(MESH_RESOLUTION, z_cov)

In [ ]:
len_xi_v = 2
len_xi_z = 2

x = fe.Point(0,0)
xi_v = np.random.uniform(-np.sqrt(3), np.sqrt(3), len_xi_v)
xi_z = np.random.normal(0, 1, len_xi_z)

mesh = mshr.generate_mesh(DOMAIN, 32)
V = fe.FunctionSpace(mesh, 'P', 1)
a_hat_function = fe.Function(V)

dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, mesh.geometry().dim()))
for i, x in enumerate(dof_coordinates):
    V_x = randomFieldV(x, xi_v)
    a_hat_value = a_hat_random_field(V_x, randomFieldV, randomFieldZ, xi_v, xi_z)
    a_hat_function.vector()[i] = a_hat_value

In [ ]:
# # Plot the function using FEniCS and Matplotlib
# plt.figure()
# c = fe.plot(a_hat_function)
# plt.title('a_hat_random_field on the unit circle')
# plt.colorbar(c)
# plt.show()


dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, mesh.geometry().dim()))
# Create a grid of points for the contour plot
perturbed_coords = np.array([randomFieldV(coords, xi_v) for coords in dof_coordinates])
x_coords = perturbed_coords[:, 0]
y_coords = perturbed_coords[:, 1]
z_values = a_hat_function.vector().get_local()

grid_x, grid_y = np.mgrid[np.min(x_coords):np.max(x_coords):500j, np.min(y_coords):np.max(y_coords):500j]
grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')


# Create the contour plot using Matplotlib
plt.figure()
contour = plt.contourf(grid_x, grid_y, grid_z, levels=20, cmap='viridis')
plt.colorbar(contour)
plt.title('Contour plot of a_hat_random_field on the unit circle')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]  # Increased resolution
grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')

# Apply a logarithmic transformation to the z-values
log_grid_z = np.log(np.abs(grid_z) + 1e-10)  # Adding a small value to avoid log(0)

# Create the contour plot using Matplotlib with a logarithmic z-scale
plt.figure()
contour = plt.contourf(grid_x, grid_y, log_grid_z, levels=120, cmap='viridis')
plt.colorbar(contour, label='log(z)')
plt.title('Logarithmic Contour plot of a_hat_random_field on the unit circle')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
mc_sample_sizes = [1, 2, 4]
len_xi_v = 8
len_xi_z = 8
mesh_resolution_NVA = 4
mesh_resolution_inverse_mapping = MESH_RESOLUTION
mesh_resolution_solution = 4


for mc_sample_size in mc_sample_sizes:
    # Maybe consider pushing out some fixed code of the loop, but for refactoring reasons it is kept here

    xis_v = [np.random.uniform(-np.sqrt(3), np.sqrt(3), len_xi_v) for _ in range(mc_sample_size)]
    xis_z = [np.random.normal(0, 1, len_xi_z) for _ in range(mc_sample_size)]

    NVA = non_varying_area(len_xi_v, randomFieldV)
    mesh_NVA = mshr.generate_mesh(NVA, mesh_resolution_NVA)

    V = fe.FunctionSpace(mesh_NVA, 'P', 1)
    dofmap = V.dofmap()
    dof_coordinates = V.tabulate_dof_coordinates().reshape((-1, mesh_NVA.geometry().dim())) # Those are all the point in mesh_NVA but ordered by the dof which we need to assign by the means

    u_P_NVA_mean = np.zeros(len(dof_coordinates))

    u_sols = []
    for xi_v, xi_z in zip(xis_v, xis_z):
        u_sols.append(solve_diffusion_poisson_for_given_sample(mesh_resolution_solution, RHS_F, randomFieldV, jacobianV, randomFieldZ, xi_v, xi_z))
        
    for i, P_coords in enumerate(dof_coordinates):
        P = fe.Point(P_coords) #! loop can be optimized because P_hat is unique for each xi_v
        for xi, u_sol in zip(xis_v, u_sols):
            P_hat = inverse_mapping(P, randomFieldV, xi, mesh_resolution_inverse_mapping)
            u_P_NVA_mean[i] += u_sol(P_hat) / mc_sample_size

    u_mean = fe.Function(V)
    u_mean.set_allow_extrapolation(True)
    u_mean.vector()[:] = u_P_NVA_mean

    # Extract the values of the function at the mesh points
    x_coords = mesh_NVA.coordinates()[:, 0]
    y_coords = mesh_NVA.coordinates()[:, 1]
    z_values = []

    for i in range(len(x_coords)):
        z_values.append(u_mean(x_coords[i], y_coords[i]))

    # Create a finer grid of points for the surface plot
    grid_x, grid_y = np.mgrid[-1:1:500j, -1:1:500j]  # Increased resolution
    grid_z = griddata((x_coords, y_coords), z_values, (grid_x, grid_y), method='linear')

    # Create the interactive surface plot using Plotly
    fig = go.Figure(data=[go.Surface(z=grid_z, x=grid_x, y=grid_y, colorscale='Viridis')])

    # Customize the plot
    fig.update_layout(title=f'Mean solution Function for {mc_sample_size} samples', autosize=True,
                    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='u(x, y)'),
                    margin=dict(l=65, r=50, b=65, t=90))
    fig.show()


In [ ]:
from helpers_diffusion import diffusion_analyse_two_resolutions_from_data_u_hat
import fenics as fe

diffusion_analyse_two_resolutions_from_data_u_hat(resolution_sparse = 10,
                                  resolution_fine = 14,
                                  P_hat = fe.Point(0.5, 0.5))

In [ ]:
from helpers import calculate_vector_field_eigenpairs
from helpers_diffusion import diffusion_sobol_calc_indices_from_data, diffusion_plot_sobols

mesh_res = 8
size_xi_v = 4
size_xi_z = 4

randomFieldV, jacobianV = calculate_vector_field_eigenpairs(mesh_res)

S_single, S_total, mc_sample_size = diffusion_sobol_calc_indices_from_data(fem_res=mesh_res, kl_res=mesh_res, size_xi_v=size_xi_v, size_xi_z=size_xi_z, randomFieldV=randomFieldV, jacobianV=jacobianV)

diffusion_plot_sobols(S_single, S_total, mc_sample_size, size_xi_v, f"Sobol Indices for Diffusion Problem Mesh resolution {mesh_res}")
